In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df=pd.read_csv("Datos/Transformados/df_limpio.csv")


In [2]:
df.head()

,ID,Edad,Ingresos,Monto_Inicial,Scoring_Crediticio,Meses_Empleo,Num_Creditos,Ratio_Interes,Duracion,Ratio_Deuda_Ingresos,...,Estudios_Escolar,Estudios_Grado Universitario,Estudios_Máster,Tipo_Jornada_Laboral_Autónomo,Tipo_Jornada_Laboral_Desempleado,Tipo_Jornada_Laboral_Jornada completa,Tipo_Jornada_Laboral_Tiempo parcial,Estado_Civil_Casado,Estado_Civil_Divorciado,Estado_Civil_Soltero
0,S97R7X,18,61628,83011,397,113,1,8.06,48,0.45,...,False,False,False,True,False,False,False,True,False,False
1,RLGTBY,50,82410,68642,486,14,3,21.96,12,0.71,...,True,False,False,False,False,False,True,False,True,False
2,SKE2P9,37,110810,101859,765,115,3,11.20,60,0.40,...,False,True,False,False,False,False,True,False,True,False
3,E2FB1D,56,16815,157551,643,108,1,21.24,24,0.72,...,False,False,True,True,False,False,False,False,False,True
4,TKSCGH,35,42315,180635,645,114,3,16.95,12,0.50,...,False,True,False,False,False,True,False,True,False,False
